In [1]:
import pandas as pd

In [2]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../Configuration"

In [3]:
import refinitiv.data as rd

In [4]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x7f8277036d30 {name='workspace'}>

In [5]:
df_companies = pd.read_csv('Refinitiv_ESG_Universe.csv', delimiter=';')

df_companies.head()

,ISIN,Name
0,DK0010244508,A P MOLLER MAERSK B
1,DE000A1TNNN5,A S CREATION TAPETEN
2,CA0002551095,A&W REVENUE RYLT.INC.FD. UTS.
3,CNE100002RY5,A-LIVING SERVICES 'H'
4,US00181T1079,A-MARK PRECIOUS METALS


In [6]:
df_companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9960 entries, 0 to 9959
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISIN    9960 non-null   object
 1   Name    9960 non-null   object
dtypes: object(2)
memory usage: 155.8+ KB


In [7]:
# example, also works with ISIN instead of BASF Ticker
df = rd.get_data(
    universe=["BASFn.DE", "IBM.N"],
    fields=["TR.HeadquartersCountry", "TR.HeadquartersRegion", "TR.NAICSNationalIndustry", "TR.NAICSIndustryGroup", 
            "TR.NAICSSector", "TR.GICSSubIndustry", "TR.GICSIndustry", "TR.ICBIndustry", "TR.ICBSector", "TR.GICSIndustryGroup", "TR.SICIndustryGroup"
            "TR.GICSSector", "TR.SICIndustry", "TR.TRBCIndustry", "TR.TRBCIndustryGroup", "TR.TRBCBusinessSector", "TR.TRBCEconomicSector",
            "TR.TSE33SectorNameMain", "TR.NAICSSubsector"]
)

df

,Instrument,Country of Headquarters,Region of Headquarters,NAICS National Industry Name,NAICS Industry Group Name,NAICS Sector Name,GICS Sub-Industry Name,GICS Industry Name,ICB Industry name,ICB Sector name,GICS Industry Group Name,SIC Industry Name,TRBC Industry Name,TRBC Industry Group Name,TRBC Business Sector Name,TRBC Economic Sector Name,TSE33 Main sector name,NAICS Subsector Name
0,BASFn.DE,Germany,Europe,Paint and Coating Manufacturing,"Paint, Coating, and Adhesive Manufacturing",Manufacturing,Diversified Chemicals,Chemicals,Basic Materials,Chemicals,Materials,,Diversified Chemicals,Chemicals,Chemicals,Basic Materials,,Chemical Manufacturing
1,IBM.N,United States of America,Americas,Computer Systems Design Services,Computer Systems Design and Related Services,"Professional, Scientific, and Technical Services",IT Consulting & Other Services,IT Services,Technology,Software & Computer Services,Software & Services,,IT Services & Consulting,Software & IT Services,Software & IT Services,Technology,,"Professional, Scientific, and Technical Services"


In [8]:
# testing the loop
test_frame = df_companies.iloc[:15, :]

# loop to collect data for all available companies, stored in a dictionary with a df for each isin
financials_data_dict = {}

for isin in test_frame["ISIN"]:

    df = rd.get_history(
        universe=isin,
        fields=["TR.RevenueMean", "TR.EarningsMean", "TR.F.EmpAvg", "TR.EPSMean", "TR.ROAMean", "TR.ROEMean",
        "TR.Volume", "TR.CompanyMarketCap", "TR.FCFMean", "TR.WACC", "TR.EBITDAMean", "TR.OrganicSalesGrowthMean"],
        interval="1Y",
        start="2000-01-01",
        end="2023-01-01",
    )
    
    df.index = pd.to_datetime(df.index)
    df = df.groupby(df.index.year).first()

    financials_data_dict[isin] = df

# Assume `dfs_dict` is the dictionary of dataframes
dfs = []
for key, df in financials_data_dict.items():
    df['key'] = key  # Add a new column containing the key
    dfs.append(df)

# Concatenate the dataframes into a single one
result = pd.concat(dfs)

result.to_csv('comp_financials_test.csv')

result.head()

,Revenue - Mean,Employees - Average,Earnings Per Share - Mean,Return On Assets - Mean,Return On Equity - Mean,Volume,Company Market Cap,Free Cash Flow - Mean,"Weighted Average Cost of Capital, (%)",EBITDA - Mean,Organic Sales Growth - Mean,key
Date,,,,,,,,,,,,
1999,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,,DK0010244508
2000,4293590660,<NA>,24.44766,<NA>,<NA>,0,65573400000.0,<NA>,<NA>,<NA>,,DK0010244508
2001,8304533230,<NA>,60.83292,<NA>,<NA>,0,54027000000.0,<NA>,<NA>,2500150330,,DK0010244508
2002,10294556440,<NA>,63.19224,<NA>,18.19,0,52014960000.0,<NA>,<NA>,1929737080,,DK0010244508
2003,26008260480,<NA>,109.89529,<NA>,17.6,0,182856959999.998993,<NA>,<NA>,6631310620,,DK0010244508


In [9]:
# loop to collect data for all available companies, stored in a dictionary with a df for each isin
financials_data_dict = {}

for isin in df_companies["ISIN"]:

    df = rd.get_history(
        universe=isin,
        fields=["TR.RevenueMean", "TR.EarningsMean", "TR.F.EmpAvg", "TR.EPSMean", "TR.ROAMean", "TR.ROEMean",
        "TR.Volume", "TR.CompanyMarketCap", "TR.FCFMean", "TR.WACC", "TR.EBITDAMean", "TR.OrganicSalesGrowthMean"],
        interval="1Y",
        start="2000-01-01",
        end="2023-01-01",
    )
    
    df.index = pd.to_datetime(df.index)
    df = df.groupby(df.index.year).first()

    financials_data_dict[isin] = df

# Assume `dfs_dict` is the dictionary of dataframes
dfs = []
for key, df in financials_data_dict.items():
    df['key'] = key  # Add a new column containing the key
    dfs.append(df)

# Concatenate the dataframes into a single one
result = pd.concat(dfs)

result.to_csv('comp_financials.csv')

result.head()

,Revenue - Mean,Employees - Average,Earnings Per Share - Mean,Return On Assets - Mean,Return On Equity - Mean,Volume,Company Market Cap,Free Cash Flow - Mean,"Weighted Average Cost of Capital, (%)",EBITDA - Mean,Organic Sales Growth - Mean,key
1999,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,,DK0010244508
2000,4293590660,<NA>,24.44766,<NA>,<NA>,0,65573400000.0,<NA>,<NA>,<NA>,,DK0010244508
2001,8304533230,<NA>,60.83292,<NA>,<NA>,0,54027000000.0,<NA>,<NA>,2500150330,,DK0010244508
2002,10294556440,<NA>,63.19224,<NA>,18.19,0,52014960000.0,<NA>,<NA>,1929737080,,DK0010244508
2003,26008260480,<NA>,109.89529,<NA>,17.6,0,182856959999.998993,<NA>,<NA>,6631310620,,DK0010244508


In [10]:
rd.close_session()